In [4]:
import pandas as pd

# selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

In [2]:
# paths settings
path_to_chromedriver = 'C:/chromedriver.exe'

# Open Browser
options = Options()
options.add_argument("--start-maximized")

browser = webdriver.Chrome(executable_path=path_to_chromedriver, options=options)

In [3]:
web = 'https://namecensus.com/zip-codes/'
browser.get(web)

#get table rows elements
table_rows = browser.find_elements_by_css_selector('body > div.container > section > div > main > div.table-expand > table > tbody > tr')

#get all the links for each state
link_dict = {}
for row in table_rows:
    state = row.find_element_by_css_selector('td:nth-child(1)').text
    link = row.find_element_by_css_selector('td:nth-child(1) > a').get_attribute('href')
    
    link_dict[state] = link
    

In [7]:
city_dict = {}
county_dict = {}

for key,val in link_dict.items():
    
    #navigate to each link
    browser.get(val)
    
    #turn off ads pop up if any
    try:
        browser.find_element_by_css_selector('#dismiss-button').click()
    except:
        pass
    
    #get table rows element
    state_table_rows = browser.find_elements_by_css_selector('body > div.container > section > div > main > div.table-expand > table > tbody > tr')
    
    #extract city and county list
    city_list = []
    county_list = []
    for r in state_table_rows:
        
        city = r.find_element_by_css_selector('td:nth-child(2)').text
        county = r.find_element_by_css_selector('td:nth-child(3)').text
        
        if city not in city_list:
            city_list.append(city)
        if county not in county_list:
            county_list.append(county)
    
    city_dict[key] = city_list
    county_dict[key] = county_list

In [8]:
#join list of city and county by separator '\n'
n_city_dict = {}
for key,val in city_dict.items():
    n_val = list(set(val)) 
    n_val = '\n'.join(n_val)
    n_city_dict[key] = n_val
    
n_county_dict = {}
for key,val in county_dict.items():
    n_val = list(set(val)) 
    n_val = '\n'.join(n_val)
    n_county_dict[key] = n_val

In [11]:
#get city dataframe
df_city = pd.DataFrame.from_dict(n_city_dict,orient='index')
df_city = df_city.reset_index()
df_city.columns = ['state', 'city']
df_city = df_city.assign(city=df_city.city.str.split('\n'))
df_city = df_city.explode('city')
df_city.reset_index(drop=True, inplace=True)
df_city

,state,city
0,Alabama,"Paint Rock, Pleasant Groves, Scottsboro, Skyli..."
1,Alabama,"Littleville, Muscle Shoals, Tuscumbia"
2,Alabama,"Hammondville, Henagar, Mentone, Valley Head"
3,Alabama,"Carbon Hill, Lynn, Nauvoo"
4,Alabama,"Henagar, Ider, Sylvania"
...,...,...
24099,Wyoming,Baggs
24100,Wyoming,"Atlantic City, Boulder Flats, Ethete, Fort Was..."
24101,Wyoming,"Afton, Fairview, Osmond, Smoot"
24102,Wyoming,"Rawlins, Sinclair"


In [12]:
#get county dataframe
df_county = pd.DataFrame.from_dict(n_county_dict,orient='index')
df_county = df_county.reset_index()
df_county.columns = ['state', 'county']
df_county = df_county.assign(county=df_county.county.str.split('\n'))
df_county = df_county.explode('county')
df_county.reset_index(drop=True, inplace=True)
df_county

,state,county
0,Alabama,"Cleburne County, Randolph County"
1,Alabama,"Cherokee County, Etowah County"
2,Alabama,Houston County
3,Alabama,"Jefferson County, Shelby County, St. Clair County"
4,Alabama,"Coffee County, Dale County"
...,...,...
8499,Wyoming,"Campbell County, Johnson County, Sheridan County"
8500,Wyoming,"Carbon County, Fremont County, Sweetwater County"
8501,Wyoming,"Goshen County, Platte County"
8502,Wyoming,Johnson County


In [14]:
#export to csv
df_city.to_csv('us_city.csv', index=False)
df_county.to_csv('us_county.csv', index=False)

In [13]:
#data source from https://www.careerbuilder.com/advice/average-salary-by-city
us_salary = pd.read_csv('us_salary.csv')
us_salary

,City,Avg. salary*
0,"San Francisco, California","$96,500"
1,"San Jose, California","$93,000"
2,"Seattle, Washington","$84,500"
3,"Boston, Massachusetts","$91,000"
4,"New York, New York","$84,000"
5,"Washington, D.C.","$85,500"
6,"Denver, Colorado","$80,000"
7,"Austin, Texas","$81,500"
8,"Houston, Texas","$78,000"
9,"Los Angeles, California","$80,000"
